In [ ]:
!pip install ffmpeg-python --quiet
!pip install pyannote.audio --quiet
!pip install torchvision==0.23.0 --quiet
# !pip install onnxruntime-gpu --quiet

In [ ]:
import os
import torch
from pyannote.audio import Pipeline

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# EDIT THIS
CLEAN_AUDIO_PATH = "english_cleaned.wav"

os.makedirs("diarization_output", exist_ok=True)
os.makedirs("segments", exist_ok=True)

print("Using cleaned audio:", CLEAN_AUDIO_PATH)

HF_TOKEN = "TOKEN"
PYANNOTE_API_KEY = "TOKEN"

Using device: cuda
Using cleaned audio: english_cleaned.wav


In [ ]:
# Pyannote Percision-2 Model

pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-precision-2",
    token=PYANNOTE_API_KEY
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Pyannote Community-1 Model

pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-community-1",
    token=HF_TOKEN
)

In [ ]:
# Pyannote 3.1 Model

pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    token=HF_TOKEN
)

In [ ]:
# Pyannote 3.0 Model

pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.0",
    token=HF_TOKEN
)

speaker-embedding.onnx:   0%|          | 0.00/26.5M [00:00<?, ?B/s]

In [ ]:
pipeline.to(device)
output = pipeline(CLEAN_AUDIO_PATH)
print("Diarization complete.")

/usr/local/lib/python3.12/dist-packages/pyannote/audio/models/blocks/pooling.py:103: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /pytorch/aten/src/ATen/native/ReduceOps.cpp:1839.)
  std = sequences.std(dim=-1, correction=1)


Diarization complete.


In [ ]:
segments = []

for turn, speaker in output.speaker_diarization:
    segments.append({
        "speaker": speaker,
        "start": float(turn.start),
        "end": float(turn.end)
    })

print("Raw segments:", len(segments))
segments[0:]

Raw segments: 37


[{'speaker': 'SPEAKER_01', 'start': 0.45284375, 'end': 1.0265937500000002},
 {'speaker': 'SPEAKER_01', 'start': 1.09409375, 'end': 3.5578437500000004},
 {'speaker': 'SPEAKER_01', 'start': 3.69284375, 'end': 4.16534375},
 {'speaker': 'SPEAKER_01', 'start': 4.3678437500000005, 'end': 7.77659375},
 {'speaker': 'SPEAKER_01',
  'start': 8.063468750000002,
  'end': 8.856593750000002},
 {'speaker': 'SPEAKER_01', 'start': 9.19409375, 'end': 11.539718750000002},
 {'speaker': 'SPEAKER_00',
  'start': 11.539718750000002,
  'end': 11.556593750000001},
 {'speaker': 'SPEAKER_01',
  'start': 12.906593750000003,
  'end': 12.923468750000001},
 {'speaker': 'SPEAKER_00', 'start': 12.923468750000001, 'end': 13.21034375},
 {'speaker': 'SPEAKER_00', 'start': 13.800968750000003, 'end': 15.47159375},
 {'speaker': 'SPEAKER_00',
  'start': 15.589718750000003,
  'end': 16.602218750000002},
 {'speaker': 'SPEAKER_00',
  'start': 16.973468750000002,
  'end': 17.395343750000002},
 {'speaker': 'SPEAKER_00',
  'start'

In [ ]:
def clean_diarization_segments(segments, gap_threshold=0.5, min_duration=0.4):

    merged = []

    for seg in segments:
        if merged and merged[-1]["speaker"] == seg["speaker"]:
            if seg["start"] - merged[-1]["end"] <= gap_threshold:
                merged[-1]["end"] = seg["end"]
            else:
                merged.append(seg)
        else:
            merged.append(seg)

    cleaned = [
        s for s in merged if s["end"] - s["start"] >= min_duration
    ]

    # force first speaker as SPEAKER_00
    if cleaned:
        first = cleaned[0]["speaker"]
        if first != "SPEAKER_00":
            for s in cleaned:
                s["speaker"] = (
                    "SPEAKER_00" if s["speaker"] != "SPEAKER_00" else "SPEAKER_01"
                )

    return cleaned


cleaned_segments = clean_diarization_segments(segments)

print("Cleaned segments:", len(cleaned_segments))
cleaned_segments[0:]

Cleaned segments: 12


[{'speaker': 'SPEAKER_00', 'start': 0.45284375, 'end': 11.539718750000002},
 {'speaker': 'SPEAKER_01',
  'start': 13.800968750000003,
  'end': 19.504718750000002},
 {'speaker': 'SPEAKER_01', 'start': 20.01096875, 'end': 21.12471875},
 {'speaker': 'SPEAKER_00', 'start': 21.46221875, 'end': 26.52471875},
 {'speaker': 'SPEAKER_01', 'start': 28.02659375, 'end': 34.962218750000005},
 {'speaker': 'SPEAKER_00', 'start': 35.31659375, 'end': 40.22721875},
 {'speaker': 'SPEAKER_01', 'start': 41.914718750000006, 'end': 49.66034375},
 {'speaker': 'SPEAKER_01', 'start': 51.04409375, 'end': 57.77721875},
 {'speaker': 'SPEAKER_00', 'start': 58.43534375, 'end': 64.00409375000001},
 {'speaker': 'SPEAKER_00', 'start': 65.35409375, 'end': 68.69534375},
 {'speaker': 'SPEAKER_00', 'start': 69.21846875, 'end': 70.18034375},
 {'speaker': 'SPEAKER_00', 'start': 71.27721875, 'end': 71.88471875}]

In [ ]:
import json

json_path = "diarization_output/noise_code_switch.json"

with open(json_path, "w") as f:
    json.dump(cleaned_segments, f, indent=4)

print("Saved diarization JSON →", json_path)

In [ ]:
import ffmpeg

for i, seg in enumerate(cleaned_segments):
    start = seg["start"]
    end = seg["end"]
    speaker = seg["speaker"]

    out_path = f"segments/seg_{i}_{speaker}.wav"

    (
        ffmpeg
        .input(CLEAN_AUDIO_PATH, ss=start, to=end)
        .output(out_path, ac=1, ar=16000)
        .overwrite_output()
        .run(quiet=True)
    )

print("Saved all segments → /segments")